In [3]:
import numpy as np
import operator
import cv2
import sys, os
import keras.models 
from IPython.display import clear_output


import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from glob import glob
from PIL import Image
from tensorflow.keras.optimizers import Adam





counter=0
list=[]

IMAGE_SIZE = [64, 64]
# add preprocessing layer to the front of VGG
base = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in base.layers:
  layer.trainable = True
 
dropout1 = Dropout(0.5)
x = base.output
x = GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x = dropout1(x) 
x=Dense(512,activation='relu')(x)
prediction = Dense(7,activation='softmax')(x)
model = Model(inputs=base.input, outputs=prediction)




model.load_weights('weights.hdf5')
cap = cv2.VideoCapture(0)


subtractor = cv2.bgsegm.createBackgroundSubtractorCNT(2,False,20,True)
while(cap.isOpened()):
    _, frame = cap.read()
    
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    #cv2.rectangle(frame, (300,65),(635,415), (255, 255, 255), 2)
    cv2.rectangle(frame, (380,100),(620,360), (255, 255, 255), 2) #top left coordinate and bottom right coordinate
    #roi = frame[65:415, 300:635]
    roi = frame[100:360, 420:620] # yt,yb and xt,xb
    roi = cv2.resize(roi, (64, 64))         
    mask = subtractor.apply(roi, -1)
    p = (cv2.countNonZero(mask)*100)/(4096) 
    mask = cv2.merge((mask,mask,mask))    
    
    cv2.imshow("mask", mask)
    # Batch of 1
    result = model.predict(mask.reshape(1, 64, 64, 3))#it gives prob of each gesture i.e. 111 55 222 414
    #print(result)
    
    prediction ={'ZERO': result[0][0], 
                  'ONE': result[0][1], 
                  'TWO': result[0][2],
                  'THREE': result[0][3],
                  'FOUR': result[0][4],
                  'FIVE': result[0][5],
                 'NOTHING': result[0][6]
                  }    #it give "name" to each prob series wise i.e zero:111 one:55.....
    
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True) #it sorts and find max and place it first i.e three:414 two:222  
    
    
    a=prediction[0][0]
    list=([a]+list)
    
    
    
    #saves results in a list and then output out number that has most frequency as "num"  
    if len(list) < 20:
        for i in list:
            curr=list.count(i)
            #print(counter)
            #print(curr)
            if(curr>counter):
                counter=curr
                num=i 
    else:
         list.clear()
         counter = 0
       
    
    # removes noisy pixels which have value less than 1.
    if(p<1):
        list.clear()
        counter=0
        num='NOTHING'

    #print only the latest result    
    print(list)        
    clear_output(wait=True)
    
    
    
    # Displaying the predictions
    cv2.putText(frame, num, (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1) #it picks first element from above series, which is actually biggest prob label   
    cv2.imshow("Frame", frame)
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
        
cap.release()
cv2.destroyAllWindows()

[]
